# Lab:  Source Localization for EEG

EEG or [Electroencephalography](https://en.wikipedia.org/wiki/Electroencephalography) is a powerful tool for neuroscientists in understanding brain activity.  In EEG, a patient wears a headset with electrodes that measures voltages at a number of points on the scalp.  These voltages arise from ionic currents within the brain.  A common *inverse problem* is to estimate the which parts of the brain caused the measured response.  Source localization is useful in understanding which parts of the brain are involved in certain tasks.  A key challenge in this inverse problem is that the number of unknowns (possible locations in the brain) is much larger than the number of measurements.  In this lab, we will use LASSO regression on a real EEG dataset to overcome this problem and determine the brain region that is active under an auditory stimulus.

In addition to the concepts in the [prostate LASSO demo](./demo_prostate.ipynb) you will learn to:
* Represent responses of multi-channel time-series data, such as EEG, using linear models
* Perform LASSO and Ridge regression
* Select the regularization level via cross-validation
* Visually compare the sparsity between different solutions

We first download standard packages.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import Lasso, Ridge
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

## Load the Data

The data in this lab is taken from one of the sample datasets in the [MNE website](https://martinos.org/mne/stable/index.html). The sample data is a recording from one subject who experienced some auditory stimulus on the left ear.    

The raw data is very large (`1.5G`) and also requires that you install the `mne` python package. To make this lab easier, I have extracted and processed a small section of the data.  The following commands will get the data from a `pickle` file `eeg_dat.p` stored remotely.

In [2]:
import cloudpickle
from urllib.request import urlopen
with urlopen("http://www.ece.ohio-state.edu/~schniter/nonpublic/eeg_dat.p") as fp:
    [X,Y] = cloudpickle.load(fp)

To understand the data, there are three key variables:
* `nt`    = number of time steps that we measure data
* `nchan` = number of channels (i.e. electrodes) measured in each time step
* `ncur`  = number of currents in the brain that we want to estimate.  

Current is measured at each brain region (called a *voxel*) in three separate directions: `x`, `y`, and `z`.  So,

    nvoxels = ncur / 3
    
The components of the `X` and `Y` matrices are:
*  `Y[i,k]` = electric field measurement on channel `i` at time `k`
*  `X[i,j]` = sensitivity of channel `i` to current `j`.

Using `X.shape` and `Y.shape` compute and print `nt`, `nchan`, `ncur` and `nvoxels`.

In [3]:
# TODO
# nt = ...
# ncur = ...
# nchan = ...
# nvoxels

## The need for Regularization

Our goal is to estimate the currents `W` in the brain from the measurements `Y`.  We will use the linear model

    Y[i,k]  = \sum_j X[i,j]*W[j,k]+ b[k]

where `W[j,k]` is the value of current `j` at time `k` and where `b[k]` is a bias.  We can solve for the current matrix `W` via linear regression.  

Note that this model differs slightly from the linear regression model that we considered in most of the lectures, in the following sense. At sample index `i`, the target is a *vector* `Y[i,:]` (not a scalar `y[i]`). Thus, the regresson coefficients `W[j,:]` must also be *vectors* (rather than a scalars `w[j]`) and the bias `b[:]` must also be a *vector* (rather than a scalar `b`).  In other words, if we considered only a *single* value of `k` in the above model, then we would get a scalar-target model that takes the form `y[i] = \sum_j X[i,j]*w[j] + b`.  But in this lab, we have several values of `k`.

However, there is a major problem:
*  There are `nt` times `ncur` unknowns in `W`
*  There are `nt` time `nchan` measurements in `Y`
*  `ncur` is much larger than `nchan`

In other words, we have

    number of unknowns >> number of measurements
    
In this case, the least-squares solution is not unique (and the matrix we'd like to invert when computing the least-squares solution is not invertible).  We can remedy this problem using regularization.  We first try Ridge regression and later LASSO.


## Ridge Regression

First split the data into training and test.  Use the `train_test_split` function with `test_size=0.33`.

In [4]:
# TODO
# Xtr,Xts,Ytr,Yts = train_test_split(...) 

Use the `Ridge` regression object in `sklearn` to fit the model on the training data.  Start with a regularization weight of `alpha=1` and see how well this works.

In [5]:
# TODO
# regr = Ridge(...)

Predict the values `Y` on both the training and test data.  Use the `r2_score` method to measure the `R^2` value on both the training and test.  You will see that `R^2` value is large for the training data, while it is very low for the test data.  This suggest that, with regularization weight `alpha=1`, the model is over-fitting the data.

In [6]:
# TODO
# rsq_tr = ...
# rsq_ts = ...

Next, try to see if we can get a better `R^2` score using a different value of `alpha`.  In particular, use cross-validation to measure the test `R^2` for 20 `alpha` values logarithmically spaced from `10^{-2}` to `10^{2}` (use `np.logspace()`).  You can use cross-validation on the train/test split from above.  You do not need to do `K`-fold.

In [7]:
# TODO

Plot the test `R^2` vs. `alpha`.  And print the maximum test `R^2`.  You should see that the maximum test `R^2` is better, but still not very high.

In [8]:
# TODO

Now, let's take a closer look at the solution. 

* Find the optimal regularization `alpha` from the above cross-validation
* Re-fit the model using the training data and the optimal `alpha` 
* Extract the matrix `W` of linear model coefficients.  These are stored as `regr.coef_` in transposed form, so apply a transpose to get W as we defined it above
* For each current `j` compute `Wrms[j] =  sqrt( sum_k W[j,k]**2 )`, which is the root mean squared value across time.
* Finally, plot the `Wrms` vector.

You will see that the RMS current vector `Wrms` is not sparse.  This means that the solution found by Ridge regression allows non-zero currents in all locations.

In [9]:
# TODO

## LASSO Regression

We can improve the estimate by rewarding sparsity.  Biologically, we know that only a limited number of brain regions should be involved in the response to a particular stimuli.  As a result, we expect that the current matrix `W[j,k]` should be zero-valued for most locations `j` at each time `k`.  We can impose this constraint using LASSO regularization with an appropriate strength `alpha`.

Re-fit the training data using the `Lasso` model with `alpha=1e-4`.  Also set `max_iter=100` and `tol=0.01`.  The LASSO solver is much slower than Ridge regression, so this may take a minute.

In [10]:
# TODO

Next, evaluate the model on the test data and measure the `R^2` value.  You should get a better fit than you did with Ridge regression.  

In [11]:
# TODO

Still, the value `alpha=1e-4` was only a guess.  We should do even better with the optimal `alpha`. 

Use cross-validation (with the above test/train split) to evaluate different values of `alpha` logarithically spaced between `alpha=1e-4` and `alpha=1e-3`.  Computing each fit may take some time, so use only 5 values of `alpha`. Also, for each `alpha` you try, store the matrix `W`.  This way, you will not have to re-fit the model if you want to examine the entries of `W`.

In [12]:
# TODO

Plot the `R^2` value vs. `alpha`.  Print the optimal `R^2`.  You should see it is higher than with the best Ridge Regression case.

In [13]:
# TODO

Display the RMS current values `Wrms` for the optimal `alpha`, as you did in the Ridge Regression case.  You will see that is much sparser.

In [14]:
# TODO

## More fun

If you want to dive deeper into this application:
* Install the [MNE python package](https://martinos.org/mne/stable/index.html).  This is an amazing package with many tools for processing EEG data.
* In particular, you can use the above to visualize where in the brain the currents are active.
* You can also improve the fitting with better regularization.  For example, you could experiment with the combination of LASSO and ridge regression, known as ElasticNet.  There is a regression object for this in sklearn.linear_model.
* We could also use a more sophisticated regularization technique called Group LASSO, that encourages *entire rows* of the `W` matrix to be zero-valued.  This is appropriate in this application because, if the current is zero for one time, it is likely to be zero for all time. 
* You can even use the recovered current patterns to make predictions about what the patient is seeing or hearing or thinking!  It sounds like science-fiction, but it has been demonstrated to work to a certain extent!